In [ ]:
# import getpass
# import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [ ]:
import random
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def weather(query: str) -> str:
    """Procura pelo clima de uma cidade online"""
    weather = ["ensolarado", "nublado", "chuvoso", "neve"]
    result = random.choice(weather)
    return result

@tool
def multiply(a: int, b: int) -> int:
    """Multiplica dois números"""
    return a * b

tools = [weather, multiply]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente mas não sabe do clima ou fazer cálculos matemáticos.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [ ]:
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "Qual o clima em Recife?"})

## Exercícios

### Exercício 1
Adicione uma função nas ferramentas do agente para criar arquivos de texto. Em seguida, teste o agente.

### Exercício 2
Crie um agente que conte a quantidade de ocorrências de uma determinada letra em uma palavra. Em seguida, teste o agente.